# <center>Formulación de un problema de optimización (Grupo 06)</center>
# <center>"Trayectoria del disparo de un cañón con objetivo"</center>


# 0. Introducción

Como parte de la segunda parte de la última práctica de la asignatura Optimización Heurística, se ha elaborado el presente Notebook como memoria de la realización del proyecto de la formulación de un problema de optimización. 

Para este segundo apartado se propone al grupo la formulación de un problema de la manera correcta para que pueda ser resuelto mediante el uso de algoritmos genéticos. Para ello será necesario cumplir los siguientes requisitos:

- La función de fitness será programada desde cero.
- Se deberá debatir los aspectos relacionados con el problema escogido (número de variables, alelos, operadores específicos...)


De esta manera, hemos decidido dividir la presente memoria en los siguientes apartados:

1. <b>Planteamiento del problema:</b> presentación del problema, descripción del mismo, variables a tener en cuenta, objetivo del problema.
2. <b>Codificación de las variables:</b> forma en la que se han cofidicado las variables.
3. <b>Función de fitness:</b> explicación de la función empleada.
4. <b>Funcionamiento del problema:</b> implementación del algoritmo con el problema y ejemplo de resolución del problema.
5. <b>Conclusión:</b> solución del problema, resultados obtenidos y comentarios finales.

# 1. Planteamiento del problema


Se requiere diseñar un cañón capaz de disparar un proyectil para que se acerque lo máximo posible a un objetivo que se encuentre en el plano horizontal (en el suelo). Para ello el cañón dispondrá de las siguientes 3 variables:

<!--<center><img src="img/canon.png" width="500px" height="350px"/></center>-->

1. Cantidad de pólvora (con un máximo de 100grs).
2. Ángulo de elevación del cañón.
3. Ángulo de rotación del cañón.

Éstas deberán ser ajustadas correctamente de cara a conseguir hacer caer el proyectil sobre la posición del objetivo.

# 2. Codificación de las variables

El problema dispone de 3 variables:

1. **Pólvora:** para realizar el disparo se dispone de un máximo de 100grs de pólvora para generar el impuslo inicial de la bala.
2. **Ángulo de elevación:** se trata del ángulo respecto al plano horizontal. Será por tánto como máximo de 90º.
3. **Ángulo de rotación:** ángulo necesario para apuntar hacia el objetivo, siendo el máximo de 360º.

Todas las variables han sido homogeneizadas entre el intervalo \[0, 1] para asegurar los bounds de todas ellas.

In [4]:
polvora = (0,1)
elevacion = (0,1)
rotacion = (0,1)

mybounds = [polvora, elevacion, rotacion]

# 3. Función de fitness

El problema por tanto debe tratar de minimizar la distancia entre la posición en el plano donde ha caído el proyectil y el objetivo.

De esta manera se tiene la siguiente función de fitness:

In [5]:
import benchmarks.functions as functions
import group06.EA as ea
import group06.Genome as genome
import math

# Función auxiliar que calcula la distancia entre dos puntos en el plano
def dist(p1, p2):
    return math.sqrt(math.pow(p1[0] - p2[0],2) + math.pow(p1[1] - p2[1], 2))

def fitness(variables):

    p_obj = (4, 9)  # Coordenadas del objetivo
    g = 9.8         # Gravedad para la simulacion
    m = 1000        # Masa de la bala de cañon
    Ec_kg = 3000000 # Energía cinética por cada Kg de polvora

    
    gr_powder =  variables[0] * 100 # Gramos de pólvora que disponemos
    angle_elev = variables[1] * 90  # Ángulo de elevación del cañón
    angle_rot =  variables[2] * 360 # Ángulo de rotación

    # Calculamos la energia cinética que se generaría con determinada cantidad de pólvora
    Ec = (Ec_kg * gr_powder) / 1000
    sqr_V0 = (2 * Ec) / m

    # Calculamos la distancia máxima que recorrería la bala
    recorrido = (sqr_V0 * math.sin(math.radians(2 * angle_elev))) / g

    # Calculamos la distancia entre el punto donde ha caido la bala y nuestro objetivo
    x = math.sin(math.radians(angle_rot)) * recorrido
    z = math.cos(math.radians(angle_rot)) * recorrido
    p_bala = (x, z)

    return dist(p_obj, p_bala)

# 4. Funcionamiento del problema

In [6]:
myea = ea.EA(fitness, mybounds, 50)
myea.run(1000)
sol = myea.best()
print(sol.solution)
print(sol.fitnes)

[0.16091451 0.49207841 0.06656247]
2.8799573918150326e-08


# 5. Conclusión